In [3]:
import pandas as pd
import numpy as np

In [21]:
outs = pd.read_csv('data/outliers_and_data_per_tid_per_date.csv', index_col=0)
incomes = pd.read_excel('data/terminal_data_hackathon v4.xlsx', sheet_name='Incomes', index_col='TID')

In [15]:
outs.head()

,406136,406139,406145,406148,406180,406190,406196,406504,406509,406536,...,699451_outl_abs_500000_sigma_0,699540_outl_abs_500000_sigma_0,699560_outl_abs_500000_sigma_0,699572_outl_abs_500000_sigma_0,699577_outl_abs_500000_sigma_0,699578_outl_abs_500000_sigma_0,699579_outl_abs_500000_sigma_0,699629_outl_abs_500000_sigma_0,699641_outl_abs_500000_sigma_0,699664_outl_abs_500000_sigma_0
2022-09-01,90000,103000,143000,50000,96000,219000,115000,382000,128000,63000,...,0,0,0,0,0,0,0,0,0,0
2022-09-02,105000,206000,136000,73000,82000,236000,123000,317000,126000,65000,...,0,0,0,0,0,0,0,0,0,0
2022-09-03,99000,168000,124000,53000,71000,218000,121000,479000,170000,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-04,107000,124000,117000,65000,72000,212000,120000,333000,113000,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-05,110000,78000,123000,75000,86000,207000,96000,232000,120000,201000,...,0,0,0,0,0,0,0,0,0,0


In [16]:
outs1 = outs.iloc[:,['_outl_abs_300000_sigma_1' in x for x in outs.columns]].sum()
outs1[outs1!=0] # более 300тыс и правее 1сигмы от средней

406139_outl_abs_300000_sigma_1     1
406145_outl_abs_300000_sigma_1     2
406190_outl_abs_300000_sigma_1     2
406196_outl_abs_300000_sigma_1     1
406504_outl_abs_300000_sigma_1    12
                                  ..
693658_outl_abs_300000_sigma_1     1
698735_outl_abs_300000_sigma_1     1
698956_outl_abs_300000_sigma_1     1
698957_outl_abs_300000_sigma_1     1
698983_outl_abs_300000_sigma_1     1
Length: 71, dtype: int64

In [17]:
outs2 = outs.iloc[:,['_outl_abs_500000_sigma_1' in x for x in outs.columns]].sum()
outs2[outs2!=0] # более 500тыс и правее 1сигмы от средней

406190_outl_abs_500000_sigma_1     1
605329_outl_abs_500000_sigma_1     9
605378_outl_abs_500000_sigma_1     1
606531_outl_abs_500000_sigma_1     1
616681_outl_abs_500000_sigma_1     4
629806_outl_abs_500000_sigma_1     1
634558_outl_abs_500000_sigma_1    10
634655_outl_abs_500000_sigma_1     1
635576_outl_abs_500000_sigma_1     1
636538_outl_abs_500000_sigma_1     1
636752_outl_abs_500000_sigma_1     1
636754_outl_abs_500000_sigma_1     1
644825_outl_abs_500000_sigma_1     2
657936_outl_abs_500000_sigma_1     1
658930_outl_abs_500000_sigma_1     2
669424_outl_abs_500000_sigma_1     1
676383_outl_abs_500000_sigma_1     1
676981_outl_abs_500000_sigma_1     1
686118_outl_abs_500000_sigma_1     1
dtype: int64

In [18]:
problem_tid_lst = [int(x[:6]) for x in outs1[outs1!=0].index]

In [19]:
pred = pd.read_csv('data/predictions_all_period.csv', index_col=0).T

In [22]:
diffs = pd.DataFrame(data=incomes.iloc[:,1:].values - pred.values, index=pred.index, columns=pred.columns)
diffs_abs = diffs.apply(abs) 
diffs2 = pd.DataFrame(diffs_abs.values /incomes.iloc[:,1:].values,
                      index=pred.index, columns=pred.columns).replace(-np.inf, 0).replace(np.inf, 0)
diffs2.mean(axis=1).describe() # все отклонения

<ipython-input-22-4e2526c70d44>:3: RuntimeWarning: divide by zero encountered in divide
  diffs2 = pd.DataFrame(diffs_abs.values /incomes.iloc[:,1:].values,
<ipython-input-22-4e2526c70d44>:3: RuntimeWarning: invalid value encountered in divide
  diffs2 = pd.DataFrame(diffs_abs.values /incomes.iloc[:,1:].values,


count    1630.000000
mean        0.213993
std         0.169214
min         0.046954
25%         0.124892
50%         0.165314
75%         0.243742
max         2.858016
dtype: float64

In [23]:
diffs3 = pd.DataFrame(diffs.values/incomes.iloc[:,1:].values,
                      index=pred.index, columns=pred.columns).replace(-np.inf, 0).replace(np.inf, 0)
diffs3 = diffs3.applymap(lambda x: max(0,x))
diffs3.mean(axis=1).describe() # только отклонения предикта в меньшую сторону

<ipython-input-23-09d9a6ff8021>:1: RuntimeWarning: divide by zero encountered in divide
  diffs3 = pd.DataFrame(diffs.values/incomes.iloc[:,1:].values,
<ipython-input-23-09d9a6ff8021>:1: RuntimeWarning: invalid value encountered in divide
  diffs3 = pd.DataFrame(diffs.values/incomes.iloc[:,1:].values,


count    1630.000000
mean        0.076577
std         0.043574
min         0.019990
25%         0.048229
50%         0.063256
75%         0.087055
max         0.455354
dtype: float64